### Diverse usage of LLMs
This notebook demonstrates how LLMs can be used in a variety of industries. Note that the examples used in this notebook are not exhaustive and there are many other industries where LLMs can be used. Note that the responses generated by LLMs are not always accurate and should be used with caution.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct",
                                             torch_dtype="auto",
                                             ).to(device).eval()

In [2]:
def get_response(messages):
    text = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

messages = [{"role": "system", 
             "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
            {"role": "user", "content": ""}]

### 1. Healthcare Data Extraction
This section demonstrates how to extract healthcare data from a conversation. The goal of this task is to extract the relevant information from the conversation and present it in a structured format.

In [3]:
conversation = """
Patient: Hello, I have a headache.
Doctor: Do you have any medication on your head?
Patient: Yes, I have aspirin.
Doctor: Do you have any allergies?
Patient: No, I don't have any.
Doctor: Do you have any family history of headaches?
Patient: No, I don't have any.
Doctor: Do you have any past medical history of headaches?
Patient: No, I don't have any.
Doctor: Do you have any recent surgery or hospitalization for headaches?
Patient: No, I don't have any.
Doctor: Do you have any recent trauma or injury?
Patient: No, I don't have any.
Doctor: Do you have any recent surgery or hospitalization for headaches?
Patient: No, I don't have any.
Doctor: Do you have any family history of headaches?
Patient: No, I don't have any.
Doctor: Do you have any allergies?
Patient: No, I don't have any.
"""

prompt = f"Extract only the key points from the conversation and summarize in a table using the markdown format. Conversation: {conversation}"
messages[1]["content"] = prompt
response = get_response(messages)
print(response)

| Question | Answer |
| --- | --- |
| Medication | Aspirin |
| Allergies | No |
| Family History | No |
| Past Medical History | No |
| Recent Surgery/Hospitalization | No |
| Trauma/Injury | No |
| Family History | No |
| Allergies | No |


### 2. Financial report automation using AI assistants
This section demonstrates how to generate a financial report automatically using AI assistants.

In [4]:
table = """
Grocery sales for January and February 2022:
| Date        | Item        | Quantity | Price Per Unit | Total Sold |
|-------------|-------------|----------|----------------|------------|
| 2022-01     | Apples      | 5        | $6.00          | $30.00     |
| 2022-01     | Oranges     | 8        | $0.90          | $7.20      |
| 2022-01     | Watermelon  | 1        | $6.00          | $6.00      |
| 2022-01     | Bananas     | 2        | $2.50          | $5.00      |
| 2022-02     | Milk        | 10       | $2.75          | $27.50     |
| 2022-02     | Apples      | 10       | $5.00          | $50.00     |
| 2022-02     | Watermelon  | 10       | $5.50          | $55.00     |
| 2022-02     | Oranges     | 8        | $0.75          | $6.00      |
| 2022-02     | Bananas     | 12       | $2.00          | $24.00     |
"""
print(table)


Grocery sales for January and February 2022:
| Date        | Item        | Quantity | Price Per Unit | Total Sold |
|-------------|-------------|----------|----------------|------------|
| 2022-01     | Apples      | 5        | $6.00          | $30.00     |
| 2022-01     | Oranges     | 8        | $0.90          | $7.20      |
| 2022-01     | Watermelon  | 1        | $6.00          | $6.00      |
| 2022-01     | Bananas     | 2        | $2.50          | $5.00      |
| 2022-02     | Milk        | 10       | $2.75          | $27.50     |
| 2022-02     | Apples      | 10       | $5.00          | $50.00     |
| 2022-02     | Watermelon  | 10       | $5.50          | $55.00     |
| 2022-02     | Oranges     | 8        | $0.75          | $6.00      |
| 2022-02     | Bananas     | 12       | $2.00          | $24.00     |



In [5]:
prompt = f"Summarize on the impact of promotion on sales for fruits on February 2022, in bullet points, based on the table:\n {table}. Ignore the items that are not fruit-related."
messages[1]["content"] = prompt
response = get_response(messages)
print(response)

- **Promotion Impact on Sales:**
  - **Apples:** Sales increased from $30.00 in January to $50.00 in February due to promotional discounts.
  - **Oranges:** Sales remained relatively stable at $7.20 in January and decreased slightly to $6.00 in February.
  - **Watermelon:** Sales increased significantly from $6.00 in January to $55.00 in February.
  - **Bananas:** Sales dropped from $5.00 in January to $24.00 in February due to promotions reducing prices.


### 3. Retail and E-Commerce, Automotive
This section demonstrates the sentiment analysis for the product reviews.

In [6]:
reviews = """
Customer1: I love this product. It is very well made and looks great.
Customer2: I bought this product for my daughter. It is very cute and soft.
Customer3: I am very disappointed with this product. It is not as good as I expected.
Customer4: I am so happy with this product. It is exactly what I was looking for.
Customer5: I am not impressed with this product. It is too small and uncomfortable to hold.
Customer6: I am not sure if this product is worth the price. It is too expensive for me.
Customer7: The product is very expensive and I don't want to spend that much.
Customer8: The product is very well made and looks great. It is exactly what I was looking for.
Customer9: I am glad that I bought this product. It is exactly what I was looking for.
"""
labels = '[pos, pos, neg, pos, neg, neg, neg, pos, pos]'

prompt = f"Provide suggestions, in bullet points, for how to improve the sales of the product based on the customer reviews: {reviews}"
messages[1]["content"] = prompt
response = get_response(messages)
print(response)

Based on the provided customer reviews, here are some suggestions for improving the sales of the product:

- **Highlight Positive Reviews**: Emphasize the positive aspects such as "I love this product" or "It is very well made and looks great."
- **Address Negative Reviews**: Acknowledge and address any negative feedback like "I am very disappointed with this product" or "The product is too small and uncomfortable to hold." Provide explanations or solutions where appropriate.
- **Simplify Pricing Information**: For customers unsure about the value ("I am not sure if this product is worth the price"), offer clear pricing information or comparisons to similar products.
- **Offer Customization Options**: If possible, suggest customization options that can make the product more appealing to potential buyers who might be dissatisfied with its current size or design.
- **Highlight Unique Selling Points**: Identify unique features or benefits of the product that could differentiate it from co

### 4. Publishing
This section demonstrates how to generate an introduction to a particular country using LLM.

In [7]:
prompt = f"Can you introduce Singapore in 4 sentences or less?"
messages[1]["content"] = prompt
response = get_response(messages)
print(response)

Singapore is an island city-state located at the southern tip of Southeast Asia. It's known for its advanced infrastructure and high-tech industries, as well as its emphasis on education and multiculturalism. The country has made significant progress in reducing poverty rates and improving living standards over recent decades. It is also home to several major financial institutions and international organizations. Despite being small in size, Singapore boasts a vibrant culture with a mix of Chinese, Malay, Indian, and European influences.


### Practice for the week
Let's replace the Qwen2.5-1.5B with a more recent model, such as the DeepSeek, and play around with the parameters. Note that you need to insert your own `<DeepSeek API keys>` to run DeepSeek in the code below.

If you don't have an API key, you can either generate one on the DeepSeek platform or try with other models such as the `LLaMA 2`. Note that `LLaMA 2` has its parameters ranged from 7B to 70B, it's recommended to check your hardware capabilites before running it.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="<DeepSeek API Key>", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)